In [ ]:
import glob
import json
import typing

import pandas as pd

In [ ]:
import config

CFG = config.Config()

In [ ]:
raw_samples: typing.List[dict] = [
    json.load(open(file, 'r'))
    for file in glob.glob(f'{CFG.data_questionary_dir}/*.json')
]
len(raw_samples)

In [ ]:
dataset: pd.DataFrame = (
    pd.json_normalize(raw_samples)
    .astype({'model': 'category'})
    .astype({'number': int})
    .set_index(['category', 'number'])
    .pipe(lambda _df: _df.assign(model=_df['model'].str.replace(':.*','', regex=True)))
    .pipe(lambda _df: _df.assign(response=_df['response'].str.extract(r'(\d)')))
    .pivot(columns='model')
)
dataset.to_parquet(f'{CFG.data_dir}/questionary.responses.parquet')
dataset.to_latex(f'{CFG.report_dir}/questionary.responses.tex')
dataset